# Attempting to make an algorithm that ranks college football teams.

### **PROCESS AND PREPARE DATA**

In [2]:
## LOAD game data AND team data INTO DATAFRAMES ##
import re
import pandas as pd
import numpy as np
# Load data into pandas DataFrame from "/lakehouse/default/" + "Files/CFB_Game_Data.csv"
games = pd.read_csv("/lakehouse/default/" + "Files/CFB_Game_Data.csv")
games['Date'] = pd.to_datetime(games.Date)
display(games)
# Load data into pandas DataFrame from "/lakehouse/default/" + "Files/CFB_Team_Data.csv"
teams = pd.read_csv("/lakehouse/default/Files/CFB_Team_Data.csv")
display(teams)

StatementMeta(, c313fdc4-2ce5-4c30-a7ef-a4f821add2fd, 4, Finished, Available)

SynapseWidget(Synapse.DataFrame, 69b7b567-969c-4ec3-8b2b-65a3ad372a54)

SynapseWidget(Synapse.DataFrame, 38693f3a-6cfa-4e25-8c96-f8afe53e332b)

In [3]:
## FILTERS TO ONLY REGULAR SZN GAMES ##
games = games.loc[games['Date'] < '2023-12-03']



StatementMeta(, c313fdc4-2ce5-4c30-a7ef-a4f821add2fd, 5, Finished, Available)

In [4]:
## SET INDEX OF games TO Rk
games.set_index("Rk", inplace = True)
games.head()


StatementMeta(, c313fdc4-2ce5-4c30-a7ef-a4f821add2fd, 6, Finished, Available)

,Wk,Date,Time,Day,Winner,Win_Pts,Loser,Lose_Pts
Rk,,,,,,,,
1,1,2023-08-26,5:30 PM,Sat,Jacksonville State,17,Texas-El Paso,14
2,1,2023-08-26,9:00 PM,Sat,Louisiana Tech,22,Florida International,17
3,1,2023-08-26,7:00 PM,Sat,Massachusetts,41,New Mexico State,30
4,1,2023-08-26,2:30 PM,Sat,(13) Notre Dame,42,Navy,3
5,1,2023-08-26,7:00 PM,Sat,San Diego State,20,Ohio,13


In [5]:
## CHECK FOR NULLS IN games ##
games.isnull().sum() # 0 nulls


StatementMeta(, c313fdc4-2ce5-4c30-a7ef-a4f821add2fd, 7, Finished, Available)

Wk          0
Date        0
Time        0
Day         0
Winner      0
Win_Pts     0
Loser       0
Lose_Pts    0
dtype: int64

In [6]:
## CHECK FOR NULLS IN teams ##
teams.isnull().sum() # 0 nulls

StatementMeta(, c313fdc4-2ce5-4c30-a7ef-a4f821add2fd, 8, Finished, Available)

Rk            0
School        0
Conf          0
W             0
L             0
Pct           0
Off_Points    0
Def_Points    0
SRS           0
SOS           0
dtype: int64

In [7]:
## ELIMINATE THE TEAMS RANKING FROM THE 'Winner' AND 'Loser' COLUMNS ##
# Code generated by Data Wrangler for pandas DataFrame #
# Replace all instances of "(n)" with "" in columns: 'Winner', 'Loser' #

def clean_data(games):
    for x in range(26):
        games['Winner'] = games['Winner'].str.replace(f"({x})", "", regex=False)
        games['Loser'] = games['Loser'].str.replace(f"({x})", "", regex=False)
    return games

games_clean = clean_data(games.copy())


StatementMeta(, c313fdc4-2ce5-4c30-a7ef-a4f821add2fd, 9, Finished, Available)

In [8]:
## ELIMINATE SPACES IN FRONT OF STRINGS IN 'Winner' AND 'Loser' COLUMNS ##

games_clean['Winner'] = games_clean['Winner'].str.strip()
games_clean['Loser'] = games_clean['Loser'].str.strip()



StatementMeta(, c313fdc4-2ce5-4c30-a7ef-a4f821add2fd, 10, Finished, Available)

In [9]:
## FIX TEAM NAMES ##

teams.loc[teams['School'] == 'LSU', 'School'] = 'Louisiana State'
teams.loc[teams['School'] == 'Pitt', 'School'] = 'Pittsburgh'
teams.loc[teams['School'] == 'UCF', 'School'] = 'Central Florida'
teams.loc[teams['School'] == 'SMU', 'School'] = 'Southern Methodist'
teams.loc[teams['School'] == 'UAB', 'School'] = 'Alabama-Birmingham'
teams.loc[teams['School'] == 'UTSA', 'School'] = 'Texas-San Antonio'
teams.loc[teams['School'] == 'BYU', 'School'] = 'Brigham Young'
teams.loc[teams['School'] == 'USC', 'School'] = 'Southern California'
teams.loc[teams['School'] == 'FIU', 'School'] = 'Florida International'
teams.loc[teams['School'] == 'JMU', 'School'] = 'James Madison'
teams.loc[teams['School'] == 'TCU', 'School'] = 'Texas Christian'
teams.loc[teams['School'] == 'UTEP', 'School'] = 'Texas-El Paso'
teams.loc[teams['School'] == 'Ole Miss', 'School'] = 'Mississippi'

StatementMeta(, c313fdc4-2ce5-4c30-a7ef-a4f821add2fd, 11, Finished, Available)

In [10]:
## ADD 'fcs_team_involved' COLUMN TO 'games_clean' DATAFRAME AND SET ALL VALUES TO NaN. ##

games_clean['fcs_team_involved'] = np.NaN


StatementMeta(, c313fdc4-2ce5-4c30-a7ef-a4f821add2fd, 12, Finished, Available)

In [11]:
## LOAD 'FCS_Teams.csv' INTO A NEW DATAFRAME 'fcs_teams' ##

import pandas as pd
# Load data into pandas DataFrame from "/lakehouse/default/" + "Files/FCS_Teams.csv"
fcs_teams = pd.read_csv("/lakehouse/default/" + "Files/FCS_Teams.csv", encoding='cp1252')
fcs_teams.drop('Index', axis = 1, inplace = True)
fcs_teams.loc[-1] = ['Texas A&M-Commerce']
fcs_teams.index = fcs_teams.index + 1
fcs_teams.loc[-1] = ['Long Island']
fcs_teams.index = fcs_teams.index + 1
fcs_teams.loc[-1] = ['Kennesaw State']

fcs_teams.loc[fcs_teams['Team'] == 'VMI', 'Team'] = 'Virginia Military Institute'
fcs_teams.loc[fcs_teams['Team'] == 'Bethune–Cookman', 'Team'] = 'Bethune-Cookman'
fcs_teams.loc[fcs_teams['Team'] == 'Nicholls', 'Team'] = 'Nicholls State'
fcs_teams.loc[fcs_teams['Team'] == 'Arkansas–Pine Bluff', 'Team'] = 'Arkansas-Pine Bluff'
fcs_teams.loc[fcs_teams['Team'] == 'Gardner–Webb', 'Team'] = 'Gardner-Webb'
fcs_teams.loc[fcs_teams['Team'] == 'Tarleton', 'Team'] = 'Tarleton State'
fcs_teams.loc[fcs_teams['Team'] == 'Houston Christian', 'Team'] = 'Houston Baptist'
fcs_teams.loc[fcs_teams['Team'] == 'Central Connecticut', 'Team'] = 'Central Connecticut State'
fcs_teams.loc[fcs_teams['Team'] == 'Saint Francis (PA)', 'Team'] = 'St. Francis (PA)'
fcs_teams.loc[fcs_teams['Team'] == 'UC Davis', 'Team'] = 'California-Davis'
fcs_teams.loc[fcs_teams['Team'] == 'UT Martin', 'Team'] = 'Tennessee-Martin'
fcs_teams.loc[fcs_teams['Team'] == 'McNeese', 'Team'] = 'McNeese State'
fcs_teams.loc[fcs_teams['Team'] == 'The Citadel', 'Team'] = 'Citadel'


display(fcs_teams)


StatementMeta(, c313fdc4-2ce5-4c30-a7ef-a4f821add2fd, 13, Finished, Available)

SynapseWidget(Synapse.DataFrame, 4e906912-f91f-491c-a782-2d81232631fa)

In [25]:
## POPULATE 'fcs_team_involved' COLUMN ##

games_clean['fcs_team_involved'] = games_clean['fcs_team_involved'].fillna(False)

for team in fcs_teams['Team']:    
        games_clean.loc[team == games_clean['Loser'],
            'fcs_team_involved'] = True
        games_clean.loc[team == games_clean['Winner'],
            'fcs_team_involved'] = True


games_clean['fcs_team_involved'].value_counts()


StatementMeta(, c313fdc4-2ce5-4c30-a7ef-a4f821add2fd, 27, Finished, Available)

fcs_team_involved
False    723
True     144
Name: count, dtype: int64

In [26]:
## MAKES NEW DATAFRAME fbs_games THAT ELIMINATES GAMES INVOLVING AN FCS TEAM ##
## we only want to include games involving two D1 FBS teams ##
fbs_games = games_clean.loc[games_clean['fcs_team_involved']==False]
fbs_games['fcs_team_involved'].value_counts()
fbs_games.drop('fcs_team_involved', axis = 1, inplace = True)




StatementMeta(, c313fdc4-2ce5-4c30-a7ef-a4f821add2fd, 28, Finished, Available)

/tmp/ipykernel_5866/1317939191.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fbs_games.drop('fcs_team_involved', axis = 1, inplace = True)


In [27]:
## UPDATES WINS, LOSSES, AND PCT ##

gf = fbs_games.value_counts('Winner').rename_axis('Team').reset_index(name = 'Wins')
gf2 = fbs_games.value_counts('Loser').rename_axis('Team').reset_index(name = 'Losses')
gf.head()

for team in teams['School']:
   row = gf.loc[gf.Team == team]
   if row.empty:
      teams.loc[teams['School'] == team, 'Reg_W'] = 0
   else:   
      teams.loc[teams['School'] == team, 'Reg_W'] = row.iloc[0]['Wins']
      
   #Some teams dont show up in losses df
   row2 = gf2.loc[gf2.Team == team]
   if row2.empty:
      teams.loc[teams['School'] == team, 'Reg_L'] = 0
   else:
      teams.loc[teams['School'] == team, 'Reg_L'] = row2.iloc[0]['Losses']

teams['Reg_Pct'] = (teams['Reg_W']) / (teams['Reg_W'] + teams['Reg_L'])
teams = teams.round(3)

StatementMeta(, c313fdc4-2ce5-4c30-a7ef-a4f821add2fd, 29, Finished, Available)

In [28]:
## ADD 'score_diff' COLUMN TO fbs_games DATAFRAME ##
fbs_games['score_diff'] = fbs_games['Win_Pts'] - fbs_games['Lose_Pts']
fbs_games.head()

StatementMeta(, c313fdc4-2ce5-4c30-a7ef-a4f821add2fd, 30, Finished, Available)

/tmp/ipykernel_5866/2939921415.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fbs_games['score_diff'] = fbs_games['Win_Pts'] - fbs_games['Lose_Pts']


,Wk,Date,Time,Day,Winner,Win_Pts,Loser,Lose_Pts,score_diff
Rk,,,,,,,,,
2,1,2023-08-26,9:00 PM,Sat,Louisiana Tech,22,Florida International,17,5
5,1,2023-08-26,7:00 PM,Sat,San Diego State,20,Ohio,13,7
6,1,2023-08-26,8:00 PM,Sat,Southern California,56,San Jose State,28,28
10,2,2023-08-31,7:00 PM,Thu,Central Florida,56,Kent State,6,50
12,2,2023-08-31,8:00 PM,Thu,Minnesota,13,Nebraska,10,3


### **ANALYZE DATA TO MAKE A RANKING ALGORITHM**

In [29]:
## RANKING ALGORITHM THAT ONLY TAKES INTO ACCOUNT WINS/LOSSES AND STRENGTH OF SCHEDULE ##
## ONLY USES GAMES FROM THE REGULAR SEASON, WHERE BOTH TEAMS INVOLVED WERE FBS TEAMS ##
## jt_rating = (SUM of [opponent win pcts. + 0.1] for each game won) - (SUM of [1.1 - opponent win pcts.] for each game lost) ##

Team_Rankings = teams.copy()
Team_Rankings['jt_rating'] = 0

for team in Team_Rankings.School:
    df = fbs_games.loc[team == fbs_games['Winner']]
    df2 = fbs_games.loc[team == fbs_games['Loser']]
    for loser in df['Loser']:
        row = Team_Rankings.loc[Team_Rankings['School'] == loser]
        lpct = row.iloc[0]['Reg_Pct']
        df.loc[df['Loser'] == loser, 'lpct'] = lpct
        df['pts'] = (0.1 + df['lpct']) 
        Team_Rankings.loc[Team_Rankings['School'] == team, 'jt_rating'] = df['pts'].sum()
    for winner in df2['Winner']:
        row2 = Team_Rankings.loc[Team_Rankings['School'] == winner]
        wpct = row2.iloc[0]['Reg_Pct']
        df2.loc[df2['Winner'] == winner, 'wpct'] = wpct
        df2['pts'] = ((1.1 - df2['wpct']))
        Team_Rankings.loc[Team_Rankings['School'] == team, 'jt_rating'] -= df2['pts'].sum()

Team_Rankings.sort_values('jt_rating', ascending = False).head(25)


StatementMeta(, c313fdc4-2ce5-4c30-a7ef-a4f821add2fd, 31, Finished, Available)

/tmp/ipykernel_5866/1535050762.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['Loser'] == loser, 'lpct'] = lpct
/tmp/ipykernel_5866/1535050762.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pts'] = (0.1 + df['lpct'])
/tmp/ipykernel_5866/1535050762.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

,Rk,School,Conf,W,L,Pct,Off_Points,Def_Points,SRS,SOS,Reg_W,Reg_L,Reg_Pct,jt_rating
93,94,Washington,Pac-12,14,1,0.933,36.0,24.8,17.89,7.83,13.0,0.0,1.000,7.913
42,43,Michigan,Big Ten (East),15,0,1.000,35.9,10.4,25.01,5.61,13.0,0.0,1.000,7.814
112,113,Alabama,SEC (West),12,2,0.857,34.0,19.0,18.16,6.58,11.0,1.0,0.917,6.762
0,1,Florida State,ACC,13,1,0.929,34.6,19.3,16.51,2.94,11.0,0.0,1.000,6.650
31,32,Texas,Big 12,12,2,0.857,35.8,18.9,20.32,6.61,11.0,1.0,0.917,6.486
105,106,Georgia,SEC (East),13,1,0.929,40.1,15.6,20.00,3.21,11.0,1.0,0.917,5.963
56,57,Liberty,CUSA,13,1,0.929,38.3,24.3,9.18,-4.75,12.0,0.0,1.000,5.935
43,44,Ohio State,Big Ten (East),11,2,0.846,30.5,11.2,20.26,5.64,10.0,1.0,0.909,5.713
119,120,James Madison,Sun Belt (East),11,2,0.846,34.1,19.5,8.53,-3.01,10.0,1.0,0.909,4.855
94,95,Oregon,Pac-12,12,2,0.857,44.2,16.5,19.94,3.30,10.0,2.0,0.833,4.730


In [31]:
## FIRST ATTEMPT AT ALGORITHM ##
## ATTEMPTS TO USE SCORE DIFFERENTIAL, WHICH OVERVALUES 
## BLOWOUT WINS AGAINST BAD TEAMS AND UNDERVALUES CLOSE WINS AGAINTS GOOD TEAMS ##
teams['jt_rating'] = 0

for team in teams.School:
    df = fbs_games.loc[team == fbs_games['Winner']]
    df2 = fbs_games.loc[team == fbs_games['Loser']]
    for loser in df['Loser']:
        row = teams.loc[teams['School'] == loser]
        lpct = row.iloc[0]['Pct']
        df.loc[df['Loser'] == loser, 'lpct'] = lpct
        df['pts'] = (df['score_diff']) * (df['lpct']) 
        teams.loc[teams['School'] == team, 'jt_rating'] = df['pts'].sum()
    for winner in df2['Winner']:
        row2 = teams.loc[teams['School'] == winner]
        wpct = row2.iloc[0]['Pct']
        df2.loc[df2['Winner'] == winner, 'wpct'] = wpct
        df2['pts'] = (df2['score_diff']) * ((1 - df2['wpct']))
        teams.loc[teams['School'] == team, 'jt_rating'] -= df2['pts'].sum()
    
Team_Rankings.sort_values('jt_rating', ascending = False).head(25)



StatementMeta(, c313fdc4-2ce5-4c30-a7ef-a4f821add2fd, 33, Finished, Available)

/tmp/ipykernel_5866/255451594.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['Loser'] == loser, 'lpct'] = lpct
/tmp/ipykernel_5866/255451594.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pts'] = (df['score_diff']) * (df['lpct'])
/tmp/ipykernel_5866/255451594.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

,Rk,School,Conf,W,L,Pct,Off_Points,Def_Points,SRS,SOS,Reg_W,Reg_L,Reg_Pct,jt_rating
93,94,Washington,Pac-12,14,1,0.933,36.0,24.8,17.89,7.83,13.0,0.0,1.000,7.913
42,43,Michigan,Big Ten (East),15,0,1.000,35.9,10.4,25.01,5.61,13.0,0.0,1.000,7.814
112,113,Alabama,SEC (West),12,2,0.857,34.0,19.0,18.16,6.58,11.0,1.0,0.917,6.762
0,1,Florida State,ACC,13,1,0.929,34.6,19.3,16.51,2.94,11.0,0.0,1.000,6.650
31,32,Texas,Big 12,12,2,0.857,35.8,18.9,20.32,6.61,11.0,1.0,0.917,6.486
105,106,Georgia,SEC (East),13,1,0.929,40.1,15.6,20.00,3.21,11.0,1.0,0.917,5.963
56,57,Liberty,CUSA,13,1,0.929,38.3,24.3,9.18,-4.75,12.0,0.0,1.000,5.935
43,44,Ohio State,Big Ten (East),11,2,0.846,30.5,11.2,20.26,5.64,10.0,1.0,0.909,5.713
119,120,James Madison,Sun Belt (East),11,2,0.846,34.1,19.5,8.53,-3.01,10.0,1.0,0.909,4.855
94,95,Oregon,Pac-12,12,2,0.857,44.2,16.5,19.94,3.30,10.0,2.0,0.833,4.730
